# Visualising CAMS Aerosol Forecasts

## overview

This notebook provides tools to access and visualise [Copernicus Atmosphere Monitoring Service (CAMS)](https://atmosphere.copernicus.eu/) forecasts. The data is accessed through the [Atmosphere Data Store (ADS)](https://ads.atmosphere.copernicus.eu/) and visualised via the [earthkit package](https://earthkit.readthedocs.io/).

It demonstrates how to:
- Download CAMS forecast data from the ADS
- Process and analyse the data
- Plot global maps in different projections

The notebook should contain all necessary code and default settings to exactly reproduce the below forecast chart:

![CAMS web chart to reproduce](../data/webcharts/latest_webchart.png)

## environment setup

#### conda setup
First, download and install Anaconda:

1. Visit the [Anaconda download page](https://www.anaconda.com/download)
2. Download the appropriate installer for your operating system (Windows, macOS, or Linux)
3. Run the installer and follow the installation instructions

After installing Anaconda, open a terminal (or Anaconda Prompt on Windows) and follow these steps:

First, create a new conda environment:
```bash
conda create -n cams-viz python=3.10
```

Activate the environment:
```bash
conda activate cams-viz
```

Install required packages:
```bash
conda install -c conda-forge earthkit-data earthkit-plots matplotlib numpy netcdf4 jupyterlab
```

Navigate to the directory containing this notebook and start a JupyterLab server:
```bash
jupyter lab
```

This will open a web browser where you can interact with the notebook.

#### import packages

In [ ]:
# Now we can import the required packages:
import os
import numpy as np
import earthkit.data as ekdata
import earthkit.plots as ekplots
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta

#### user settings

In [ ]:
# Data directory where files will be stored
DATA_DIR = './data'

# Get yesterday and today dates for default forecast times
today = datetime.now()
yesterday = today - timedelta(days=1)

# Forecast base time (when the forecast starts from)
# Default: Yesterday, but can be changed to any date in the past (YYYY-MM-DD)
FORECAST_BASE_TIME = yesterday.strftime('%Y-%m-%d')

# Forecast valid time (the time we want to forecast for)
# Default: Today, but can be changed to any date within 5 days of the forecast base time (YYYY-MM-DD)
FORECAST_VALID_TIME = today.strftime('%Y-%m-%d')

# Create data directory if it doesn't exist
os.makedirs(DATA_DIR, exist_ok=True)

## data download

The `CDS Application Program Interface (CDS API)` is a Python library which allows you to access data from the ADS `programmatically`. The library is available for both Python versions, Python 2.7.x and Python 3, but we recommend to use the library under Python 3. In order to use the CDS API, follow the steps below:

* [Self-register](https://ads.atmosphere.copernicus.eu/#!/home) at the ADS registration page (if you do not have an account yet)
* [Login](https://ads.atmosphere.copernicus.eu/user/login) to the ADS portal and go to the [api-how-to page](https://ads.atmosphere.copernicus.eu/api-how-to)
* Copy the CDS API key displayed in the black terminal window in a file under `$HOME/.cdsapirc`

**Note:** You find your CDS API key displayed in the black terminal box under the section `Install the CDS API key`. If you do not see a URL or key appear in the black terminal box, please refresh your browser tab. 
  

In [ ]:
# Install the CDS API client to download data:
!pip install cdsapi

In [ ]:

dataset = "cams-global-atmospheric-composition-forecasts"
request = {
    'variable': ['total_aerosol_optical_depth_550nm'],
    'date': [f'{FORECAST_BASE_TIME}/{FORECAST_BASE_TIME}'],
    'time': ['12:00'],
    'leadtime_hour': ['0'],
    'type': ['forecast'],
    'data_format': 'netcdf4',
}

client = cdsapi.Client(url=URL, key=KEY)
client.retrieve(dataset, request).download(f'{DATA_DIR}/{FORECAST_BASE_TIME}_total_aerosol_optical_depth_550nm.zip')

## processing

let's do some processing

In [ ]:
# Regrid data to 1-degree resolution
data = ekdata.regrid(data, resolution=1.0) 

In [ ]:
# Calculate time average
data = data.mean(dim='time') 

## visualisation

In [ ]:
plot_settings = {'layer_name': 'composition_aod550', 'title': 'Aerosol optical depth at 550 nm (provided by CAMS, the Copernicus Atmosphere Monitoring Service)', 'description': 'Aerosol optical depth at 550 nm. Note that this layer is ONLY available from 00UTC run around 20UTC up to step 120.', 'style_name': 'sh_all_aod', 'levels': [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.8, 1.0, 10.0], 'colours': [(0.0, 0.0, 0.9456), (0.0, 0.3, 1.0), (0.0, 0.6922, 1.0), (0.1613, 1.0, 0.8065), (0.4902, 1.0, 0.4775), (0.8065, 1.0, 0.1613), (1.0, 0.7705, 0.0), (1.0, 0.4074, 0.0), (0.9456, 0.0298, 0.0), (0.5, 0.0, 0.0)]}

In [ ]:
# Create global map visualization
fig = plt.figure(figsize=(12, 8))
ekplots.geomap(
    data,
    projection='PlateCarree',
    colormap='viridis',
    title='Global Aerosol Optical Depth'
)
plt.show() 

In [ ]:
# Create zonal mean plot
data.mean(dim='longitude').plot(
    figsize=(10, 6),
    title='Zonal Mean AOD'
)
plt.show() 